In [1]:
import pandas as pd
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.metrics import mean_squared_error

# Load your dataset
df = pd.read_csv("train.csv", parse_dates=["date"])
df.sort_values("date", inplace=True)

# Prepare a list to collect results
all_forecasts = []

# Limit to fewer items/stores for faster testing (optional)
stores = df['store'].unique()[:2]    # change [:2] to all if needed
items = df['item'].unique()[:2]

# Loop over each store-item pair
for store_id in stores:
    for item_id in items:
        subset = df[(df['store'] == store_id) & (df['item'] == item_id)].copy()
        subset.set_index('date', inplace=True)
        subset = subset.asfreq('D').fillna(method='ffill')  # ensure daily frequency

        # Train-test split
        split_date = '2017-01-01'
        train = subset.loc[:split_date]
        test = subset.loc[split_date:]

        try:
            model = ExponentialSmoothing(
                train['sales'],
                trend='add',
                seasonal='add',
                seasonal_periods=365
            ).fit()

            forecast = model.forecast(len(test))

            result_df = pd.DataFrame({
                'date': test.index,
                'store': store_id,
                'item': item_id,
                'actual_sales': test['sales'].values,
                'predicted_sales': forecast.values
            })

            all_forecasts.append(result_df)

        except Exception as e:
            print(f"Skipping Store {store_id}, Item {item_id} due to error: {e}")

# Combine all forecast results
final_df = pd.concat(all_forecasts)

# Save to CSV
final_df.to_csv("sales_forecast_with_categories.csv", index=False)

print("✅ Forecast data with categories saved to 'sales_forecast_with_categories.csv'")


✅ Forecast data with categories saved to 'sales_forecast_with_categories.csv'


In [2]:
# Save the forecast to a CSV file again
final_df.to_csv("sales_forecast_with_categories.csv", index=False)

# Check the full path
import os
print("Looking in:", os.getcwd())  # This shows your current working directory

# Confirm absolute file path
print("Full file path:", os.path.abspath("sales_forecast_with_categories.csv"))


Looking in: C:\Users\dhair\Untitled Folder
Full file path: C:\Users\dhair\Untitled Folder\sales_forecast_with_categories.csv
